In [5]:
#importing libraries
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
import bs4 as BeautifulSoup
import urllib.request  

In [6]:
# !pip install nltk
# !pip install bs4

In [33]:
#fetching the content from the URL
fetched_data = urllib.request.urlopen('https://en.wikipedia.org/wiki/Hair')

In [34]:
article_read = fetched_data.read()
# article_read

In [35]:
#parsing the URL content and storing in a variable
article_parsed = BeautifulSoup.BeautifulSoup(article_read,'html.parser')
article_parsed

<!DOCTYPE html>

<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>Hair - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRequestId":"XknsVwpAICAAAH2LJdYAAAEI","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Hair","wgTitle":"Hair","wgCurRevisionId":940506235,"wgRevisionId":940506235,"wgArticleId":14313,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["CS1 maint: location","Webarchive template wayback links","Pages with URL errors","CS1 German-language sources (d

In [37]:
#returning <p> tags
paragraphs = article_parsed.find_all('p')
paragraphs

[<p class="mw-empty-elt">
 </p>,
 <p><b>Hair</b> is a <a href="/wiki/Protein_filament" title="Protein filament">protein filament</a> that grows from <a href="/wiki/Hair_follicle" title="Hair follicle">follicles</a> found in the <a href="/wiki/Dermis" title="Dermis">dermis</a>. Hair is one of the defining characteristics of <a href="/wiki/Mammal" title="Mammal">mammals</a>.
 The human body, apart from areas of glabrous skin, is covered in follicles which produce thick <a href="/wiki/Terminal_hair" title="Terminal hair">terminal</a> and fine <a href="/wiki/Vellus_hair" title="Vellus hair">vellus hair</a>. Most common interest in hair is focused on <a class="mw-redirect" href="/wiki/Hair_growth" title="Hair growth">hair growth</a>, hair types, and <a href="/wiki/Hair_care" title="Hair care">hair care</a>, but hair is also an important <a href="/wiki/Biomaterial" title="Biomaterial">biomaterial</a> primarily composed of protein, notably <a href="/wiki/Alpha-keratin" title="Alpha-keratin">a

In [38]:
#looping through the paragraphs and adding them to the variable
article_content = ''
for p in paragraphs:  
    article_content += p.text


In [39]:
article_content

'\nHair is a protein filament that grows from follicles found in the dermis. Hair is one of the defining characteristics of mammals.\nThe human body, apart from areas of glabrous skin, is covered in follicles which produce thick terminal and fine vellus hair. Most common interest in hair is focused on hair growth, hair types, and hair care, but hair is also an important biomaterial primarily composed of protein, notably alpha-keratin.\nAttitudes towards different forms of hair, such as hairstyles and hair removal, vary widely across different cultures and historical periods, but it is often used to indicate a person\'s personal beliefs or social position, such as their age, sex, or religion.[1]\nThe word "hair" usually refers to two distinct structures: \nHair fibers have a structure consisting of several layers, starting from the outside:\nEach strand of hair is made up of the medulla, cortex, and cuticle.[4] The innermost region, the medulla, is not always present and is an open, uns

In [20]:
def _create_dictionary_table(text_string) -> dict:
   
    #removing stop words
    stop_words = set(stopwords.words("english"))
    
    words = word_tokenize(text_string)
    
    #reducing words to their root form
    stem = PorterStemmer()
    
    #creating dictionary for the word frequency table
    frequency_table = dict()
    for wd in words:
        wd = stem.stem(wd)
        if wd in stop_words:
            continue
        if wd in frequency_table:
            frequency_table[wd] += 1
        else:
            frequency_table[wd] = 1

    return frequency_table

In [21]:
def _calculate_sentence_scores(sentences, frequency_table) -> dict:   

    #algorithm for scoring a sentence by its words
    sentence_weight = dict()

    for sentence in sentences:
        sentence_wordcount = (len(word_tokenize(sentence)))
        sentence_wordcount_without_stop_words = 0
        for word_weight in frequency_table:
            if word_weight in sentence.lower():
                sentence_wordcount_without_stop_words += 1
                if sentence[:7] in sentence_weight:
                    sentence_weight[sentence[:7]] += frequency_table[word_weight]
                else:
                    sentence_weight[sentence[:7]] = frequency_table[word_weight]

        sentence_weight[sentence[:7]] = sentence_weight[sentence[:7]] / sentence_wordcount_without_stop_words

       

    return sentence_weight

In [22]:
def _calculate_average_score(sentence_weight) -> int:
   
    # Calculating the average score for the sentences
    sum_values = 0
    for entry in sentence_weight:
        sum_values += sentence_weight[entry]

    # Getting sentence average value from source text
    average_score = (sum_values / len(sentence_weight))

    return average_score

In [23]:
def _get_article_summary(sentences, sentence_weight, threshold):
    sentence_counter = 0
    article_summary = ''

    for sentence in sentences:
        if sentence[:7] in sentence_weight and sentence_weight[sentence[:7]] >= (threshold):
            article_summary += " " + sentence
            sentence_counter += 1

    return article_summary

In [24]:
def _run_article_summary(article):
    
    #creating a dictionary for the word frequency table
    frequency_table = _create_dictionary_table(article)

    #tokenizing the sentences
    sentences = sent_tokenize(article)

    #algorithm for scoring a sentence by its words
    sentence_scores = _calculate_sentence_scores(sentences, frequency_table)

    #getting the threshold
    threshold = _calculate_average_score(sentence_scores)

    #producing the summary
    article_summary = _get_article_summary(sentences, sentence_scores, 1.5 * threshold)

    return article_summary

In [40]:
if __name__ == '__main__':
    summary_results = _run_article_summary(article_content)
    print(summary_results)

 The only "living" portion of the hair is found in the follicle. Hair exists in a variety of textures. Three main aspects of hair texture are the curl pattern, volume, and consistency. The derivations of hair texture are not fully understood. There are a range of theories pertaining to the curl patterns of hair. This means the bonds present are directly in line with one another, resulting in straight hair. While the circumference of the hair follicle expands, so does the thickness of the hair follicle. Medium hair feels like a cotton thread, but isn't stiff or rough. It is neither fine nor coarse. Coarse hair feels hard and wiry. The function of hair in other locations is debated. On average, men have more body hair than women. [52] For instance, vellus hair is a juvenile characteristic. tightly coiled). Shaving is accomplished with bladed instruments, such as razors. Cut hair may be used in wigs. Hair color and texture can be a sign of ethnic ancestry. Facial hair is a sign of puberty

In [26]:
article_content

'The 20th (twentieth) century was a century that began on\nJanuary 1, 1901[1] and ended on December 31, 2000.[2] It was the tenth and final century of the 2nd millennium. Strictly speaking, it is distinct from the century known as the 1900s which began on January 1, 1900, and ended on December 31, 1999.\nThe 20th century was dominated by a chain of events that heralded significant changes in world history as to redefine the era: flu pandemic, World War I and World War II, nuclear power and space exploration, nationalism and decolonization, the Cold War and post-Cold War conflicts; intergovernmental organizations and cultural homogenization through developments in emerging transportation and communications technology; poverty reduction and world population growth, awareness of environmental degradation, ecological extinction;[3][4] and the birth of the Digital Revolution, enabled by the wide adoption of MOS transistors and integrated circuits. It saw great advances in power generation, 

In [41]:
content2="""Text summarization is the technique for generating a concise and precise summary of voluminous texts while focusing on the sections that convey useful information, and without losing the overall meaning.Automatic text summarization aims to transform lengthy documents into shortened versions, something which could be difficult and costly to undertake if done manually. Machine learning algorithms can be trained to comprehend documents and identify the sections that convey important facts and information before producing the required summarized texts. For example, the image below is of this news article that has been fed into a machine learning algorithm to generate a summary."""

In [29]:
content2

'Text summarization is the technique for generating a concise and precise summary of voluminous texts while focusing on the sections that convey useful information, and without losing the overall meaning.\nAutomatic text summarization aims to transform lengthy documents into shortened versions, something which could be difficult and costly to undertake if done manually.\nMachine learning algorithms can be trained to comprehend documents and identify the sections that convey important facts and information before producing the required summarized texts. For example, the image below is of this news article that has been fed into a machine learning algorithm to generate a summary.'

In [43]:
_run_article_summary(content2)

''

In [44]:
content3="""A neural network is a series of algorithms that endeavors to recognize underlying relationships in a set of data through a process that mimics the way the human brain operates. In this sense, neural networks refer to systems of neurons, either organic or artificial in nature. Neural networks can adapt to changing input; so the network generates the best possible result without needing to redesign the output criteria. The concept of neural networks, which has its roots in artificial intelligence, is swiftly gaining popularity in the development of trading systems.
Basics of Neural Networks
Neural networks, in the world of finance, assist in the development of such process as time-series forecasting, algorithmic trading, securities classification, credit risk modeling and constructing proprietary indicators and price derivatives.

A neural network works similarly to the human brain’s neural network. A “neuron” in a neural network is a mathematical function that collects and classifies information according to a specific architecture. The network bears a strong resemblance to statistical methods such as curve fitting and regression analysis.

A neural network contains layers of interconnected nodes. Each node is a perceptron and is similar to a multiple linear regression. The perceptron feeds the signal produced by a multiple linear regression into an activation function that may be nonlinear.

In a multi-layered perceptron (MLP), perceptrons are arranged in interconnected layers. The input layer collects input patterns. The output layer has classifications or output signals to which input patterns may map. For instance, the patterns may comprise a list of quantities for technical indicators about a security; potential outputs could be “buy,” “hold” or “sell.”

Hidden layers fine-tune the input weightings until the neural network’s margin of error is minimal. It is hypothesized that hidden layers extrapolate salient features in the input data that have predictive power regarding the outputs. This describes feature extraction, which accomplishes a utility similar to statistical techniques such as principal component analysis."""

In [47]:
content3=content3.replace('\n',' ')
content3

'A neural network is a series of algorithms that endeavors to recognize underlying relationships in a set of data through a process that mimics the way the human brain operates. In this sense, neural networks refer to systems of neurons, either organic or artificial in nature. Neural networks can adapt to changing input; so the network generates the best possible result without needing to redesign the output criteria. The concept of neural networks, which has its roots in artificial intelligence, is swiftly gaining popularity in the development of trading systems.  Basics of Neural Networks Neural networks, in the world of finance, assist in the development of such process as time-series forecasting, algorithmic trading, securities classification, credit risk modeling and constructing proprietary indicators and price derivatives.  A neural network works similarly to the human brain’s neural network. A “neuron” in a neural network is a mathematical function that collects and classifies 

In [48]:
_run_article_summary(content3)

' A neural network is a series of algorithms that endeavors to recognize underlying relationships in a set of data through a process that mimics the way the human brain operates. A neural network works similarly to the human brain’s neural network. A neural network contains layers of interconnected nodes. The input layer collects input patterns.'